In [2]:
import torch, lab_utils, random
from torchvision.datasets import CIFAR10 
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn 
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import json, string
%matplotlib inline

In [ ]:
# Define the data loader.

In [ ]:
# This is a mofified VGG network with deconvolutional layers added.
class vgg_deconv(nn.Module):
    
    def __init__(self, n_classes=21, batch_norm = 0.1):
        super(vgg_deconv, self).__init__()
        
        seg_vgg11 = models.vgg11(pretrained = True)
        
        # Convolutional layers.
        self.conv1 = seg_vgg11.features[0]
        self.bn1 = nn.BatchNorm2d(64,momentum=batch_norm)
        self.conv2 = seg_vgg11.features[3]
        self.bn2 = nn.BatchNorm2d(128,momentum=batch_norm)
        self.conv3 = seg_vgg11.features[6]
        self.bn3 = nn.BatchNorm2d(256,momentum=batch_norm)
        self.conv4 = seg_vgg11.features[8]
        self.bn4 = nn.BatchNorm2d(256,momentum=batch_norm)
        self.conv5 = seg_vgg11.features[11]
        self.bn5 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.conv6 = seg_vgg11.features[13]
        self.bn6 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.conv7 = seg_vgg11.features[16]
        self.bn7 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.conv8 = seg_vgg11.features[18]
        self.bn8 = nn.BatchNorm2d(512,momentum=batch_norm)
        
        # Deconvolutional layers.
        self.deconv8 = nn.ConvTranspose2d(512,512,3)
        self.deconv_bn8 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.deconv7 = nn.ConvTranspose2d(512,512,3)
        self.deconv_bn7 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.deconv6 = nn.ConvTranspose2d(512,512,3)
        self.deconv_bn6 = nn.BatchNorm2d(512,momentum=batch_norm)
        self.deconv5 = nn.ConvTranspose2d(512,256,3)
        self.deconv_bn5 = nn.BatchNorm2d(256,momentum=batch_norm)
        self.deconv4 = nn.ConvTranspose2d(256,256,3)
        self.deconv_bn4 = nn.BatchNorm2d(256,momentum=batch_norm)
        self.deconv3 = nn.ConvTranspose2d(256,128,3)
        self.deconv_bn3 = nn.BatchNorm2d(128,momentum=batch_norm)
        self.deconv2 = nn.ConvTranspose2d(128,64,3)
        self.deconv_bn2 = nn.BatchNorm2d(64,momentum=batch_norm)
        self.deconv1 = nn.ConvTranspose2d(64,n_classes,3)
        self.deconv_bn1 = nn.BatchNorm2d(n_classes,momentum=batch_norm)
        
    def forward(self, x):
        # Convolutional part.
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out,2,stride=2)
        out = F.relu(self.bn2(self.conv2(out)))
        out = F.max_pool2d(out,2,stride=2)
        out = F.relu(self.bn3(self.conv3(out)))
        out = F.relu(self.bn4(self.conv4(out)))
        out = F.max_pool2d(out,2,stride=2)
        out = F.relu(self.bn5(self.conv5(out)))
        out = F.relu(self.bn6(self.conv6(out)))
        out = F.max_pool2d(out,2,stride=2)
        out = F.relu(self.bn7(self.conv7(out)))
        out = F.relu(self.bn8(self.conv8(out)))
        
        # Deconvolutional part.
        out = F.relu(self.deconv_bn8(self.deconv8(out)))
        out = F.relu(self.deconv_bn7(self.deconv7(out)))
        out = F.max_unpool2d(out,2,stride=2)
        out = F.relu(self.deconv_bn6(self.deconv6(out)))
        out = F.relu(self.deconv_bn5(self.deconv5(out)))
        out = F.max_unpool2d(out,2,stride=2)
        out = F.relu(self.deconv_bn4(self.deconv4(out)))
        out = F.relu(self.deconv_bn3(self.deconv3(out)))
        out = F.max_unpool2d(out,2,stride=2)
        out = F.relu(self.deconv_bn2(self.deconv2(out)))
        out = F.max_unpool2d(out,2,stride=2)
        out = F.relu(self.deconv_bn1(self.deconv1(out)))

In [ ]:
import gc
import numpy as np
from tqdm import tqdm as tqdm

# These are used to record loss and accuracy data for further plots.
loss_train = np.zeros(30)
loss_validate = np.zeros(30)
acc_train = np.zeros(30)
acc_validate = np.zeros(30)

# Define the training function.
def train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 10, use_gpu = False):
    if use_gpu:
        #network = torch.nn.DataParallel(network, device_ids=[0, 1])
        network = network.cuda()
        criterion = criterion.cuda()
        
    # Training loop.
    for epoch in range(0, n_epochs):
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        
        gc.collect()

        # Make a pass over the training data.
        t = tqdm(trainLoader, desc = 'Training epoch %d' % epoch)
        network.train()  # This is important to call before training!
        for (i, (inputs, labels)) in enumerate(t):
            
            gc.collect()

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # Backward pass:
            optimizer.zero_grad()
            # Loss is a variable, and calling backward on a Variable will
            # compute all the gradients that lead to that Variable taking on its
            # current value.
            loss.backward() 

            # Weight and bias updates.
            optimizer.step()

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
        
        # Record data for plotting figures.
        loss_train[epoch] = cum_loss / (1 + i)
        acc_train[epoch] = 100 * correct / counter
        
        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        t = tqdm(valLoader, desc = 'Validation epoch %d' % epoch)
        network.eval()  # This is important to call before evaluating!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        # Record data for plotting figures.
        loss_validate[epoch] = cum_loss / (1 + i)
        acc_validate[epoch] = 100 * correct / counter

In [ ]:
# Train the neural network.
learningRate = 0.01  # Feel free to change this.
set_momentum = 0.5
decay = 5e-3

# Definition of our network.
network = MyNetwork()

# Feel free to use a different loss here.
criterion = nn.CrossEntropyLoss()

# Feel free to change the optimizer, or the optimizer parameters. e.g. momentum, weightDecay, etc.
optimizer = optim.SGD(network.parameters(), lr = learningRate, momentum = set_momentum, weight_decay = decay)

# Train the previously defined model.
train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 30, use_gpu = True)

# Plot loss and accuracy.
ep = np.arange(1, 31, 1)
plt.plot(ep, loss_train, 'ro',ep, loss_validate, 'bs')
plt.xlabel('epoch')
plt.ylabel('average loss')
plt.legend(['training','validation'])
plt.show()
plt.plot(ep, acc_train, 'ro',ep, acc_validate, 'bs')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['training','validation'])
plt.show()

In [18]:
# Load and modify pre-trained VGG-11 and add deconvolutional layers.
seg_vgg11 = models.vgg11(pretrained = True)

print(seg_vgg11)

conv11 = seg_vgg11.features[0]

print(conv11)

VGG (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU (inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU (inplace)
    (15): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (16): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU (inpla